# Atelier sur la génétique statistique et l'épidémiologie génétique STAGE-Québec
## Thème 2 - Phénotypes moléculaires en épidémiologie génétique

Par Marc-André Legault (Université de Montréal) et Qihuang Zhang (Université McGill)

**31 juillet - 1er août 2025**

### Introduction

Dans ce cahier, nous utiliserons un deuxième logiciel TWAS appelé FUSION sur les mêmes statistiques sommaires GWAS. Ceci nous permettra de comprendre comment exécuter le logiciel FUSION, et de comparer les résultats avec S-PrediXcan qui est le thème de la 3e section.

### Exploration des données

La structure de fichiers pour les données FUSION incluses dans ce cahier est la suivante :
<pre>
/data/FUSION/
├── gwas/      # Statistiques sommaires GWAS
├── ld/        # Panels de référence LD (1000 Genomes EUR)
└── models/    # Modèles de poids d'expression génique
    ├── GTExv8_EUR_Pancreas/
    └── YFS_BLOOD_RNAARR/
</pre>

Chaque sous-dossier ``models/`` contient :

- Des fichiers de modèles de poids *.wgt.RDat
- Un fichier .pos listant tous les gènes disponibles

### Exécution du logiciel

Le code ci-dessous appellera FUSION pour le premier modèle de prédiction d'expression génique. Ce modèle a été dérivé de l'expression génique dans le sang total de l'étude Young Finns Study. La boucle appellera FUSION pour chaque chromosome. Pour des raisons computationnelles, vous pourriez vouloir exécuter seulement les deux premiers chromosomes. Si vous voulez exécuter tout le transcriptome, vous pouvez commenter l'instruction ``break`` en ajoutant le signe dièse au début de la ligne (``# break``).

Nous avons préparé un point de contrôle avec les résultats complets pour économiser les ressources computationnelles.

In [ ]:
for chrom in $(seq 1 22); do
    Rscript /workshop/software/FUSION/FUSION.assoc_test.R \
        --sumstats /workshop/data/FUSION/gwas/Mahajan.NatGen2022.DIAMANTE-EUR.ldsc_ss.FUSION_ref.tsv \
        --weights /workshop/data/FUSION/models/YFS_BLOOD_RNAARR/YFS.BLOOD.RNAARR.pos \
        --weights_dir /workshop/data/FUSION/models/YFS_BLOOD_RNAARR/ \
        --ref_ld_chr /workshop/data/FUSION/ld/1000G.EUR. \
        --chr $chrom \
        --out results/fusion_YFS_$chrom.tsv
        
    if (( $chrom > 2 )); then
        # Commentez la ligne ci-dessous pour exécuter tous les chromosomes,
        # sinon seulement les deux premiers seront utilisés.
        break
    fi
done

Maintenant, nous pouvons aussi exécuter FUSION avec des poids estimés à partir de GTEx (pancréas). Les mêmes données GTEx ont été utilisées pour le modèle S-PrediXcan que nous avons précédemment exécuté. Ce sera un point de comparaison entre les deux approches.

In [ ]:
for chrom in $(seq 1 22); do
    Rscript /workshop/software/FUSION/FUSION.assoc_test.R \
        --sumstats /workshop/data/FUSION/gwas/Mahajan.NatGen2022.DIAMANTE-EUR.ldsc_ss.FUSION_ref.tsv \
        --weights /workshop/data/FUSION/models/GTExv8_EUR_Pancreas/GTExv8.EUR.Pancreas.pos \
        --weights_dir /workshop/data/FUSION/models/GTExv8_EUR_Pancreas \
        --ref_ld_chr /workshop/data/FUSION/ld/1000G.EUR. \
        --chr $chrom \
        --out results/fusion_GTEx_pancreas_$chrom.tsv
        
    if (( $chrom > 2 )); then
        # Commentez la ligne ci-dessous pour exécuter tous les chromosomes,
        # sinon seulement les deux premiers seront utilisés.
        break
    fi
done

Pour plus de commodité, vous pouvez combiner les résultats en un seul fichier.

In [ ]:
# Fusionner les fichiers (GTEx Pancréas).
# cp results/fusion_GTEx_pancreas_1.tsv results/fusion_GTEx_pancreas.tsv
# for chrom in $(seq 2 22); do
#     cat results/fusion_GTEx_pancreas_${chrom}.tsv | sed 1d >> results/fusion_GTEx_pancreas.tsv
# done
#
# Fusionner les fichiers (YFS).
# cp results/fusion_YFS_1.tsv results/fusion_YFS.tsv
# for chrom in $(seq 2 22); do
#     cat results/fusion_YFS_${chrom}.tsv | sed 1d >> results/fusion_YFS.tsv
# done

Maintenant que nous avons des résultats TWAS, vous pouvez ouvrir un fichier de résultats en double-cliquant dessus dans le navigateur de fichiers. Vous devriez voir un fichier TSV avec ces colonnes :

In [ ]:
head -n 1 /workshop/local/results/fusion_GTEx_pancreas_1.tsv | tr '\t' '\n' | nl

### Explication des résultats

Chaque fichier de sortie contiendra :

- TWAS.Z, TWAS.P : Scores Z et valeurs p pour les associations gène-trait imputées
- MODELCV.R2, MODELCV.PV : Performance prédictive par validation croisée du modèle
- Best.GWAS.Z : Signal SNP GWAS le plus fort dans la région

Voici un extrait de fichier :

In [ ]:
awk '{print $1, $4, $5, $9, $10, $11, $19, $20}' /workshop/local/results/fusion_GTEx_pancreas_1.tsv | head

In [ ]:
wc -l ./results/fusion_GTEx_pancreas.tsv
wc -l ./results/fusion_YFS.tsv

<div class="alert alert-info">
Pouvez-vous identifier les gènes dans les deux tissus qui atteignent un seuil de significativité Bonferroni à l'échelle du tissu de 0,05 / # de gènes testés par tissu ? Combien y en a-t-il ?

<!--
Solution:
awk 'NR == 1 || $20 <= 0.05/5765 { print $3, $19, $20 }' results/fusion_GTEx_pancreas.tsv
awk 'NR == 1 || $20 <= 0.05/4628 { print $3, $19, $20 }' results/fusion_YFS.tsv

# 119 significatifs pour GTEx pancréas
# 85 significatifs pour YFS
//-->

</div>

<div class="alert alert-info">
Comment MODELCV.R2 affecte-t-il votre confiance ?
</div>

<div class="alert alert-info">
Certains signaux géniques sont-ils plus forts dans un tissu que dans un autre ?
</div>